In [35]:
import os 
import json 
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY =os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY =os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID=os.getenv("GOOGLE_CSE_ID")

In [36]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.utilities import TextRequestsWrapper


llm = OpenAI(temperature=0)
tools = load_tools(["google-search"], ["requests_all"])

agent = initialize_agent(tools, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True)

In [1]:
import requests
from bs4 import BeautifulSoup

def fetch_titles(url: str):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Modify the selector according to the structure of CBS News website
    titles = soup.select('h4.title')
    return [title.text.strip() for title in titles]


In [5]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.llms import OpenAI
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.utilities import TextRequestsWrapper


llm = OpenAI(temperature=0)

tools = [
    # Other tools...
    Tool(
        name="Fetch titles from CBS News",
        func=fetch_titles,
        description="Fetches titles from CBS News website"
    ),
]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [ ]:
titles = agent.run("Fetch titles from CBS News website using the fetch_titles tool with the URL https://www.cbsnews.com/")


In [ ]:
from langchain.utilities import GoogleSearchAPIWrapper

search_tool = Tool(
    name="Google Search",
    func=GoogleSearchAPIWrapper().run,
    description="Search for the CBS News top stories"
)

agent = initialize_agent([search_tool], llm, agent="zero-shot-react-description", verbose=True)

search_results = agent.run("What are the titles of the top stories on [www.cbsnews.com/]")


In [45]:
from langchain.agents import Tool
from langchain.utilities import PythonREPL
python_repl = PythonREPL()

repl_tool = Tool(
    name="python_repl",
    description="Use this tool to save the news titles in the text file. Extract the titles from the search results, Save the extracted titles to a local txt file ",
    func=python_repl.run
)

In [46]:
from langchain.agents import initialize_agent

tools = [search_tool, repl_tool]



In [47]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True, max_iterations=3)

In [48]:
agent.run("find cbs news titles, and save them in the text file")



> Entering new AgentExecutor chain...
 I need to search for the news titles and save them
Action: Google Search
Action Input: CBS News top stories
Observation: Watch CBS News live and get the latest, breaking news headlines of the day for national news and world news today. U.S. breaking news: Today's top stories updated by the CBS News team. U.S. breaking news: Today's top stories updated by the CBS News team. Go to NBCNews.com for breaking news, videos, and the latest top stories in world news, business, politics, health and pop culture. Latest world news, breaking news and today's news stories updated daily from the CBS News team. ... cbsn-fusion-hollywood-writers-strike-shuts-down-top- ... The latest news from across Canada and around the world. ... U.S. sharing 'best practices' with Canada on establishing a foreign agent registry. Emmy-winning journalist Norah O'Donnell brings years of experience with CBS News to the anchor chair, offering a weeknight survey of major news storie

'Agent stopped due to iteration limit or time limit.'